# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 1712746
- Họ và tên: Nguyễn Minh Tâm

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [18]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle
# addition
import numpy as np
import sys
def drawProgressBar(percent, barLen = 20): # show percent complete https://stackoverflow.com/questions/3002085/python-to-print-out-status-bar-and-percentage
    sys.stdout.write("\r")
    progress = ""
    for i in range(barLen):
        if i < int(barLen * percent):
            progress += "="
        else:
            progress += " "
    sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
    sys.stdout.flush()

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [2]:
def valid_url(url): 
    # django url validation regex https://stackoverflow.com/questions/7160737/how-to-validate-a-url-in-python-malformed-or-not
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return (re.match(regex, url) is not None)


def get_urls(url):
    success = False
    html_context = ""
    try:
        r=requests.get(url)
        success = True
        if(r.status_code != 200 or ("text/html" not in r.headers["content-type"])):
           return []
        html_context = r.content
    except:
        success = False
        return []
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    urls = []
    soup = BeautifulSoup(html_context, "lxml") # lxml is just the parser for reading the html
    for a in soup.find_all('a', href=True):
        if(valid_url(a['href'])): # check url, vì có khi là chỉ là url header hay sub url hay url image/pdf
            urls.append(a['href'])
    return urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    counter = 0
    visited = {} # dict {url: True,...}
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_urls = get_urls(url)
        for new_url in new_urls:
            if(not hasattr(visited, new_url)):
                visited[new_url] = True
                urls.append(new_url)
                counter += 1
                if(counter >= limit):
                    return urls
        
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 200)


## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [3]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    success = False
    html_context = ""
    try:
        r=requests.get(url)
        success = True
        if(r.status_code != 200 or ("text/html" not in r.headers["content-type"])):
           return []
        html_context = r.content
    except:
        success = False
        return []
        
    soup=BeautifulSoup(html_context,"html.parser")
    text=soup.findAll(text=True)
    filtered_text=list(filter(text_filter,text))
    word_list=[]
    #TODO:
    str_punc = string.punctuation
    str_split = ''
    for idx, c in enumerate(str_punc):
        if(idx == 0):
            str_split += f"{c}"
        else:
            str_split += f"|{c}"
    
    for sentence in filtered_text:
        if(sentence not in str_punc):
            next_words = re.sub('['+string.punctuation+']', '', sentence).split() #https://www.geeksforgeeks.org/python-extract-words-from-given-string/
            word_list = word_list + next_words
    return word_list

def read_url(url, url_idx, data):
    word_list=wordList(url)
    
    #TODO
    '''Initialize value: data[w] = [[url_idx], 1]'''
                                    #list_url
    #VD: data.get("at")==None => initialize data["at"]
        # data.get("at")!=None >= add url_idx to list_url_idx (remember to check if this url_idx is in list_url_idx), 
                                # frequence+=1
    for word in word_list:
        if(data.get(word)==None):
            data[word] = [[url_idx], 1]
        else:
            # update index urls
            if url_idx not in data[word][0]:
                data[word][0].append(url_idx)
            # update count
            data[word][1] += 1
    return 1  

In [4]:
data = {}
for url_index, url in enumerate(url_list, 1):
    read_url(url, url_index, data)
    drawProgressBar((url_index+1) / len(url_list)) # just show complete percentage

[ ==================== ] 100.50%

## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mtosity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# TODO
for stopword in english_stopwords:
    data.pop(stopword, None)

In [52]:
w = np.array(data['web'][0])
m = np.array(data['mining'][0])
print(np.intersect1d(w, m))

[  1   2   3   4  15  33  34  38  39  40  41  42  45  47  51  63  76  89
 112 130 145 163 178]


## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [13]:
with open('raw_data', 'wb') as f:
    pickle.dump(data, f)

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [14]:
with open('t_data.txt', 'w') as f:
    for word, (url_list, freq) in data.items():
        print(*url_list, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [84]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### if len(ret)==0:  return urls
    ### else update ret with urls: intersection of ret and urls 
    terms = sentence.split(' ')
    urls = np.array([], dtype='uint')
    for term_index, term in enumerate(terms):
        if not None:
            term_urls = np.array(data[term][0], dtype='uint')
            if(term_index == 0):
                urls = term_urls
            else:
                urls = np.intersect1d(urls, term_urls)# and
    if(len(ret) == 0):
        return urls
    else:
        ret = np.append(ret, urls)
    return ret

def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### find urls have at least 1 term: urls
    ### update ret with urls: extend ret with urls
    terms = sentence.split(' ')
    urls = np.array([], dtype='uint')
    for term_index, term in enumerate(terms):
        if not None:
            term_urls = np.array(data[term][0], dtype='uint')
            urls = np.append(urls, term_urls) # or
    if(len(ret) == 0):
        return urls
    else:
        ret = np.append(ret, urls)
    return ret

def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have at least 1 term: urls
    ### update ret with urls: remove urls from ret 
    all_terms = data.keys()
    not_terms = sentence.split(' ')
    and_terms = [a for a in all_terms if a not in not_terms]
    # basicly or
    terms = and_terms
    urls = np.array([], dtype='uint')
    for term_index, term in enumerate(terms):
        if not None:
            term_urls = np.array(data[term][0], dtype='uint')
            urls = np.append(urls, term_urls) # or
    if(len(ret) == 0):
        return urls
    else:
        ret = np.append(ret, urls)
    return ret

In [85]:
print(andRetrieval([], 'web mining'))
print(orRetrieval([], 'web mining'))
print(notRetrieval(url_list, 'web mining'))

[  1   2   3   4  15  33  34  38  39  40  41  42  45  47  51  63  76  89
 112 130 145 163 178]
[  1   2   3   4   7   8  15  21  33  34  35  38  39  40  41  42  45  47
  48  51  53  56  63  64  66  69  76  77  79  82  89  90  92  95  98  99
 103 107 109 110 111 112 113 115 117 119 120 130 131 132 136 140 142 143
 144 145 146 148 150 152 153 163 164 165 169 173 175 176 177 178 179 181
 183 185 186 196 198 199   1   2   3   4   5   6   9  10  15  16  17  23
  33  34  36  38  39  40  41  42  45  47  49  50  51  52  62  63  65  75
  76  78  88  89  91 112 130 145 163 178 197]
[197.   1.   2. ... 197. 197. 197.]
